# Samurai Gunn map tile classification

In [1]:
import cv2 as cv
import numpy as np
from time import time
import win32gui
import win32ui
import win32con

def list_all_open_windows():
    def winEnumHandler( hwnd, ctx ):
        if win32gui.IsWindowVisible( hwnd ):
            print (hex(hwnd), win32gui.GetWindowText( hwnd ))

    win32gui.EnumWindows(winEnumHandler, None)

#list_all_open_windows()

def window_capture():
    
    hwnd = win32gui.FindWindow(None, 'Samurai Gunn')
    #hwnd = None
    
    # Getting the window's size and accounting for window screenshot borders
    #window_rect = win32gui.GetWindowRect(hwnd)
    titlebar_px = 38
    border_px = 9

    # For samurai Gunn, the non-fullscreen dimensions should be:
    w = 320
    h = 240
    
    crop_x = border_px
    crop_y = titlebar_px
    
    wDC = win32gui.GetWindowDC(hwnd)
    dcObj = win32ui.CreateDCFromHandle(wDC)
    cDC = dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0, 0), (w, h) , dcObj, (crop_x, crop_y), win32con.SRCCOPY)
    
    
    # saving screenshot to file
    # To save screenshot to file, uncomment this 
    #bmpfilenamename = "sample" + str(img_counter) + ".jpg" #set this
    #dataBitMap.SaveBitmapFile(cDC, bmpfilenamename)
    
    # Converting to format useful for opencv
    signedIntsArray = dataBitMap.GetBitmapBits(True)
    img = np.frombuffer(signedIntsArray, dtype='uint8')
    img.shape = (h, w, 4)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())
    
    # Dropping alpha channel may be useful for some applications, like cv.matchTemplate()
    # which may throw an error otherwise
    #img = img[...,:3]   # this drops alpha channel 
    
    return img

In [2]:
'''
tiles are (16 x 16)px
 x, y  : coords of the upper left corner of the tile
 sliced: int, either 0 or 1, letting us know if it's a full tile or a half tile at the 
         upper/lower edges of the map. 
         0 means it's a full tile
         1 means it's a half tile
         
 returns a single int that will classify the tile
'''
step = 2
def classify_tile(x, y, img, sliced):
    total = np.zeros((1, 4), dtype='uint32')
    #print('total type ' + str(type(total)))
    
    corrected_height_range = int(16/(1+sliced))
    for i in range(0, corrected_height_range, step):
        for j in range(0, 16, step):
            #print('pixel type ' + str(type(img[x+i][y+j])))
            #print(x, i, x+i)
            #print(img[x+i][y+j]) #testing
            total += img[x+i][y+j]
    
    result = total * (1/(16*16/(1+sliced)))
    #result = result.astype('uint8')
    
    return result
    

In [3]:
'''
The tiles in every map can be aligned with a 20 x 15 grid of (16 x 16)px cells
img: input image
offset: in case the map tiles do not align perfectly with the grid, an offset of 8px down does 
        the trick. (Maps with different offsets have yet to be found, 
        this should allow for different offsets in the X direction)
'''
def simplify(img, offset):
    sliced = 0
    simple_img = np.zeros((15, 20, 4))
    for x in range(15):
        #print('x = ' + str(x))
        if offset == 8 and (x == 0 or x == 19):
            sliced = 1
        else:
            sliced = 0
        for y in range(20):
            # pass the coordinates of each tile with corrections to accomodate for the grid offset
            simple_img[x, y] = classify_tile(x*16 - (offset*(1-sliced)), 
                                             y*16 - (offset*(1-sliced)), 
                                             img, sliced)
            #print(simple_img[x, y])
    return simple_img

In [7]:
counter = 1
total = 0

# offset will depend on the map
# for Ice cube, offset is 0
offset = 0

while(True):
    prev_time = time()
    
    screenshot = window_capture()
    
    img = simplify(screenshot, offset)
    dim = (img.shape[1] * 16, img.shape[0] * 16)
    resized = cv.resize(img, dim, interpolation = cv.INTER_AREA) #  interpolation = cv.INTER_AREA
    cv.imshow('Simplified', resized)
    cv.imshow('Screenshot', screenshot)
    #print('FPS {}'.format(1 / (time() - prev_time)))
     
    total += 1 / (time() - prev_time)
    print('Avg FPS {}'.format(total/counter))
    counter += 1
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break

Avg FPS 4.042145516328834
Avg FPS 4.397080108864648
Avg FPS 4.695316526972857
Avg FPS 5.078138870118674
Avg FPS 5.3818148159220645
Avg FPS 5.480087185284148
Avg FPS 5.547052475260508
Avg FPS 5.656920143303463
Avg FPS 5.781036301532332
Avg FPS 5.858255809452582
Avg FPS 5.963118058985254
Avg FPS 6.0159031664298555
Avg FPS 6.06694074840485
Avg FPS 6.117283939985453
Avg FPS 6.112143966106237
Avg FPS 6.149102158099472
Avg FPS 6.1536921977064365
Avg FPS 6.1907601844742075
Avg FPS 6.21211698257138
Avg FPS 6.227054753436602
Avg FPS 6.238677550537442
Avg FPS 6.262987442356674
Avg FPS 6.254754566058772
Avg FPS 6.272663344800013
Avg FPS 6.272914995063559
Avg FPS 6.288689936413461
Avg FPS 6.274207667040675
Avg FPS 6.284124540770132
Avg FPS 6.269627678755073
Avg FPS 6.295369328148334
Avg FPS 6.300247306640982
Avg FPS 6.312206651369397
Avg FPS 6.309819356521381
Avg FPS 6.3085154213985195
Avg FPS 6.307467218309807
Avg FPS 6.313059266168908
Avg FPS 6.316178630496624
Avg FPS 6.32707917109627
Avg FPS 6.

Avg FPS 6.478018126712141
Avg FPS 6.476248159246924
Avg FPS 6.475352878458163
Avg FPS 6.47410739804715
Avg FPS 6.474471755347043
Avg FPS 6.473180453953429
Avg FPS 6.4709797848979065
Avg FPS 6.469928505701209
Avg FPS 6.467063531192741
Avg FPS 6.465533441063147
Avg FPS 6.466198182055116
Avg FPS 6.466464057859143
Avg FPS 6.465856823029609
Avg FPS 6.463718193221753
Avg FPS 6.462772410827297
Avg FPS 6.460047318666617
Avg FPS 6.458150483423855
Avg FPS 6.456687440986003
Avg FPS 6.455553340954228
Avg FPS 6.451916915706019
Avg FPS 6.449913533007642
Avg FPS 6.449124755901614
Avg FPS 6.448698758557531
Avg FPS 6.446959891063807
Avg FPS 6.445000777191579
Avg FPS 6.444769023500294
Avg FPS 6.444718362002773
Avg FPS 6.443437052655825
Avg FPS 6.441087807675337
Avg FPS 6.441408697064494
Avg FPS 6.441731719844136
Avg FPS 6.442421377729757
Avg FPS 6.440234316806059
Avg FPS 6.441866856869344
Avg FPS 6.442912666731309
Avg FPS 6.444192303954583
Avg FPS 6.444835207050244
Avg FPS 6.4453939848481925
Avg FPS 6.4